### Speed Humps Analysis (2011-2017)

#### Database: DOTDEVRHPGSQL01
#### Tables: speed_humps

#### 1. Year with most speed humps
#### 2. All mfts with either sip corridor or intersection projects of study year range
#### 3. Writing all sip project mfts of study year range from sip database to crash database
#### 4. Sipless Single Year MFT Speed Humps of study year
#### 5. Control Group: All mfts with no sips, no Speed humps and outside of study year range 

In [1]:
from ris import pysqldb
from IPython.display import clear_output
from collections import defaultdict
from collections import defaultdict, namedtuple
import datetime 
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

print 'Notebook run: {}'.format(timestamp)
print os.getcwd()

Notebook run: 2020-01-02_12:37
E:\RIS\Staff Folders\Samuel\Requests\AG\Speed Humps


In [3]:
#Database connection
cdb = pysqldb.DbConnect(server='DOTDEVRHPGSQL01', database='CRASHDATA', type='PG')
sdb = pysqldb.DbConnect(server='dotpgsql01', database='sip', type='PG')
sqldb = pysqldb.DbConnect(server='dot55sql01', database='forms', type='MS', user = 'arcgis', password = 'arcgis')


Additional database connection details required:
User name (crashdata):soge
Password (crashdata)········
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-01-02 12:37:34

Additional database connection details required:
User name (sip):soge
Password (sip)········
Database connection (PG) to sip on dotpgsql01 - user: soge 
Connection established 2020-01-02 12:37:37
	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly

Database connection (MS) to forms on dot55sql01 - user: arcgis 
Connection established 2020-01-02 12:37:37
Query run in 4000 microseconds
	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly



In [4]:
my_study_year = 2014
study_year_min = my_study_year-3
study_year_max = my_study_year+3

### 1. Year with most speed humps

In [15]:
result = cdb.dfquery("""    

SELECT left(date_installed::text,4) as year, count(distinct lion.mft)
FROM public.speed_humps sh
join archive."18d.2019-11-13_lion" lion
on sh.segmentid = lion.segmentid 
where left(date_installed::text,4)::int between 2008 and 2014
group by year
order by year

    """)

In [16]:
result

,year,count
0,2008,129
1,2009,85
2,2010,153
3,2011,106
4,2012,221
5,2013,265
6,2014,361


### 2. All mfts with either sip corridor or intersection projects of study year range

In [64]:
sip_projs = sdb.dfquery("""

--All corr/itx mfts with completed sips btwn {study_year_min}-{study_year_max}

select * from (

--All geometries that have mfts with completed sips between {study_year_min}-{study_year_max}
SELECT distinct coalesce(lion.mft,0)::text mft, coalesce(masteridfr,0)::text masteridfrom, coalesce(masteridto,0)::text masteridto, lion.geom
FROM public.sip_projects sp
join public.sip_projects_geo spg
on sp.pid=spg.pid_fk
join clion lion
on spg.segmentid::int = lion.segmentid::int
where sp.status= '15'
and spg.nodeid=0
and left(sip_year::text,4)::int between {study_year_min} and {study_year_max}
and left(end_date::text,4)::int between {study_year_min} and {study_year_max}

) sip_corr_mfts


union 

select * from (


--Completed Sip masterids to mft/geometries between {study_year_min}-{study_year_max}
with data as(


SELECT sp.pid, sp.pjct_name, sp.pm, sp.sip_year, sp.start_date, sp.end_date, spg.nodeid, st_setsrid(spg.geom,2263) geom
FROM public.sip_projects sp
join public.sip_projects_geo spg
on sp.pid=spg.pid_fk
where sp.status= '15'
and spg.segmentid=0
and left(sp.sip_year::text,4)::int between {study_year_min} and {study_year_max}
and left(sp.end_date::text,4)::int between {study_year_min} and {study_year_max}
)


SELECT distinct coalesce(clfr.mft,0)::text mft, coalesce(masteridfr,0)::text masteridfrom, coalesce(masteridto,0)::text masteridto, clfr.geom from data
left join clion_node cln
on data.nodeid::int = cln.nodeid::int
left join  clion clfr
on cln.masterid = clfr.masteridfr

union 

SELECT distinct coalesce(clto.mft,0)::text mft, coalesce(masteridfr,0)::text masteridfrom, coalesce(masteridto,0)::text masteridto, clto.geom from data
left join clion_node cln
on data.nodeid::int = cln.nodeid::int
left join clion clto
on cln.masterid = clto.masteridto


) sip_itx_mfts

""".format(study_year_min=study_year_min,study_year_max=study_year_max) 
                       )

### 3. Writing all sip project mfts of study year range from sip database to crash database

In [65]:
cdb.connect()
cdb.dataframe_to_table_schema(sip_projs, 'sip_projs', schema='working', overwrite = True)

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-01-16 12:08:31


[['mft', 'varchar (500)'],
 ['masteridfrom', 'varchar (500)'],
 ['masteridto', 'varchar (500)'],
 ['geom', 'varchar (500)']]

In [66]:
cdb.connect()
input_schema = cdb.dataframe_to_table_schema(sip_projs, 'sip_projs', schema='working', overwrite = True)
cdb.dataframe_to_table(sip_projs, 'sip_projs', input_schema, schema= "working")

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-01-16 12:08:34
Reading data into Database



83it [00:01, 49.83it/s]

Failure:

- Query run 2020-01-16 12:08:36.948000
	
                INSERT INTO working.sip_projs ("mft", "masteridfrom", "masteridto", "geom")
                VALUES ('0', '0', '0', '0105000020D70800000100000001020000000F0000000004B7EB6F462E410032EEBE099D05410050BC4E67462E41001670A1969C0541B9CCB33F4B462E41E2B2E0C84E9B05410F96587D2A462E415C6F89FD0D9A0541802CD42305462E41004C3153D5980541FEFE334CEA452E41AD7BBE0B029805414B85A32FD2452E4163E6F88F29970541F95339DEBC452E4197D252704C96054180D22E66AA452E41002057406B950541BCA7A2069A452E41BAB81DB1949405418ADD19548C452E4157D92527BB93054100492E5781452E410080082BDF920541175359D57C452E41553D15931192054156AEA3087C452E41866A81374391054180AA1CF27E452E410000CC2975900541')
            
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection closed 2020-01-16 12:08:36
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-01-16 12:08:36


406it [00:08, 49.15it/s]

Failure:

- Query run 2020-01-16 12:08:43.862000
	
                INSERT INTO working.sip_projs ("mft", "masteridfrom", "masteridto", "geom")
                VALUES ('0', '0', '0', '0105000020D70800000100000001020000000F00000000A483D245B62E410028E0CCE40F0C4158D77B3712B62E41B17739726F0F0C413C07B088DCB52E41259C87170A0F0C41F3A05915A5B52E418A5A5E52B50E0C4175EC4D2F6CB52E4140F8D79F710E0C41BF47852A32B52E41EA9ADC633F0E0C41CA3A9F5CF7B42E4180D78EE81E0E0C41001B641CBCB42E41004ADE5D100E0C41B93940C180B42E418F2A45D9130E0C417236D1A245B42E41E4859E55290E0C41CF4E56180BB42E41450035B3500E0C4147673478D1B32E414F33F0B7890E0C41468B761799B32E41806AAA0FD40E0C41AF6B504962B32E4139D2AC4C2F0F0C410095A35E2DB32E41006251E89A0F0C41')
            
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection closed 2020-01-16 12:08:43
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-01-16 12:08:43


...

Failure:

- Query run 2020-01-16 12:12:55.150000
	
                INSERT INTO working.sip_projs ("mft", "masteridfrom", "masteridto", "geom")
                VALUES ('9008919', '58000', '58059', '0105000020D70800000100000001020000001100000000584DD0B9882F41001A4BF7C1B00F41F90CA16FBE882F41646389E77AB10F4108F701C9BE882F41614475C234B20F41B501AEDBBA882F417E187CF4EDB20F41AF2E2CB0B2882F41CFF979EBA4B30F418D113A58A6882F41D7F9221A58B40F41074AA5EE95882F41F3D561FB05B50F416950119781882F41E3D5A415ADB50F418012AA7D69882F4100AA11FE4BB60F41A0D8C3D64D882F413EB2915BE1B60F41D27B6ADE2E882F41447BD6E96BB70F4193E1DDD70C882F41FBA0027CEAB70F419E6E000DE8872F4164EB3EFF5BB80F418397B6CDC0872F41191B0F7DBFB80F417E65396F97872F416223691D14B90F412C685D4B6C872F41B6478A2859B90F4100A6CFBF3F872F41002286088EB90F41')
            
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection closed 2020-01-16 12:12:55
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection establis

12564it [04:27, 46.86it/s]

Failure:

- Query run 2020-01-16 12:13:02.776000
	
                INSERT INTO working.sip_projs ("mft", "masteridfrom", "masteridto", "geom")
                VALUES ('94218', '57324', '57327', '0105000020D70800000100000001020000001C000000009863BBBB702F41008AA8D623CD0D41009B2B54E4702F41002050EA26CB0D4100AE8C57E4702F41005C9EBB26CB0D4180C734DE18712F41006EE27132C90D41800158E818712F4100E83A1A32C90D41800461F218712F4100D0F2C931C90D4100F5B34859712F410066869E4CC70D418089195159712F4100C0595B4CC70D4180C33C5B59712F4100D4BA1E4CC70D41800ED90EA5712F4100A2CC7F7CC50D4180C9001EA5712F41001A25287CC50D4100F3CB2EA5712F410004DDD77BC50D41802BCA83FB712F4100E2201FC8C30D4100B0D792FB712F4100EE6FCEC7C30D41807E60A5FB712F410090B584C7C30D4180343F975B722F4100182B4335C20D41005E0AA85B722F4100DE07F934C20D4180631CBC5B722F410038DBB534C20D41804AA401C4722F4100B2C0D3C8C00D410009D51AC4722F41002A197CC8C00D4100DA6637C4722F410014D12BC8C00D410097A25633732F41002CB28486BF0D4100EA387833732F4100EC7C2686BF0D410050309D33732F4100423ECF8

12631it [04:29, 46.89it/s]



12588 rows added to working.sip_projs



In [90]:
pysqldb.pg_to_sql(cdb, sqldb, 'speed_hump_focus_group', org_schema = 'working',print_cmd=True)


    ogr2ogr -overwrite -update -f MSSQLSpatial "MSSQL:server=dot55sql01;database=forms;UID=arcgis;PWD=arcgis" 
    PG:"host=DOTDEVRHPGSQL01 port=5432 dbname=CRASHDATA user=soge password=Dot4819!" 
    working.speed_hump_focus_group -lco OVERWRITE=yes -lco SCHEMA=dbo  -progress 
    --config MSSQLSPATIAL_USE_GEOMETRY_COLUMNS NO
    


### 4. Sipless Single Year MFT Speed Humps of Study Year

In [17]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-01-02 11:37:17


In [20]:
study_group = cdb.dfquery("""



--All no sip {my_study_year} Data

--drop table if exists working.speed_hump_focus_group; 

--create table working.speed_hump_focus_group as 

with data1 as(

--selecting all speed hump data from {my_study_year}
select distinct mft, masteridfrom, masteridto, date_installed, on_st,from_st, to_st, lion.geom, lion.segmentid 
from v_clean_speed_humps sh
join archive."18d.2019-11-13_lion" lion
on sh.segmentid = lion.segmentid 
where left(date_installed::text,4)::int = {my_study_year}
)

, data2 as(

select distinct coalesce(lion.mft,0) mft, coalesce(lion.masteridfrom,0) masteridfrom, coalesce(lion.masteridto,0) masteridto, lion.geom
from v_clean_speed_humps sh
join archive."18d.2019-11-13_lion" lion
on sh.segmentid = lion.segmentid 
where left(date_installed::text, 4)::int between {study_year_min} and {study_year_max}
and left(date_installed::text, 4)::int != {my_study_year}
)

--selecting all data that does not intersect with any other data of other years. 
select distinct data1.mft, data1.masteridfrom, data1.masteridto, lion.segmentid::int, lion.nodeidfrom::int, lion.nodeidto::int, on_st,from_st, to_st 
from data1 
join archive."18d.2019-11-13_lion" lion
on data1.segmentid = lion.segmentid 
where data1.mft not in(select mft from data2)
        and data1.masteridfrom not in ((select masteridfrom mid from data2)
                      union 
                     (select masteridto mid from data2))
        and data1.masteridto not in   ((select masteridfrom mid from data2)
                      union 
                     (select masteridto mid from data2))
        and data1.mft not in (select mft::int from working.sip_projs)
        and data1.masteridfrom not in ((select masteridfrom::int from working.sip_projs)
                      union 
                      (select masteridto::int from working.sip_projs))
        and data1.masteridto not in   ((select masteridfrom::int from working.sip_projs)
                      union 
                      (select masteridto::int from working.sip_projs));

--grant all on working.speed_hump_focus_group to public;

--select * from working.speed_hump_focus_group; 

    
    """.format(my_study_year=my_study_year,study_year_min=study_year_min,study_year_max=study_year_max) 
    )



### Study Group Nodeids

In [12]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-01-02 14:44:17


In [18]:
study_group_nodes = cdb.dfquery("""

select distinct nodeidfrom "NODEID" from working.speed_hump_study_group

union 

select distinct nodeidto "NODEID" from working.speed_hump_study_group



"""
)

In [28]:
study_group_nodes.to_csv('study_group_nodes.csv')

In [66]:
cdb.connect()
cdb.dataframe_to_table_schema(focus_group, 'speed_hump_focus_group', schema='working', overwrite = True)

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2019-12-27 17:40:27


[['mft', 'bigint'],
 ['masteridfrom', 'bigint'],
 ['masteridto', 'bigint'],
 ['segmentid', 'bigint'],
 ['nodeidfrom', 'bigint'],
 ['nodeidto', 'bigint'],
 ['on_st', 'varchar (500)'],
 ['from_st', 'varchar (500)'],
 ['to_st', 'varchar (500)']]

In [67]:
cdb.connect()
input_schema = cdb.dataframe_to_table_schema(focus_group, 'speed_hump_focus_group', schema='working', overwrite = True)
cdb.dataframe_to_table(focus_group, 'speed_hump_focus_group', input_schema, schema= "working")

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2019-12-27 17:40:54
Reading data into Database



388it [00:08, 43.31it/s]



388 rows added to working.speed_hump_focus_group



### 5. Control Group: 
#### All mfts that do not intersect with any mfts that have speed humps installed within study year range

In [61]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2019-12-27 17:36:40


In [69]:
control_group = cdb.dfquery("""



--No sip control group

drop table if exists working.speed_hump_control_group; 

create table working.speed_hump_control_group as 

with data as (


select distinct coalesce(lion.mft,0) mft, coalesce(lion.masteridfrom,0) masteridfrom, coalesce(lion.masteridto,0) masteridto, lion.geom
from v_clean_speed_humps sh
join archive."18d.2019-11-13_lion" lion
on sh.segmentid = lion.segmentid 
where left(date_installed::text, 4)::int between {study_year_min} and {study_year_max}

)



select distinct mft, masteridfrom, masteridto, segmentid::int, nodeidfrom::int, nodeidto::int
from archive."18d.2019-11-13_lion" lion
where mft not in (select mft from data)
  and masteridfrom not in ((select masteridfrom mid from data)
                            union 
                           (select masteridto mid from data))
  and masteridto not in   ((select masteridfrom mid from data)
                            union 
                           (select masteridto mid from data))
  and mft not in (select mft::int from working.sip_projs)
  and masteridfrom not in ((select masteridfrom::int from working.sip_projs)
                            union 
                           (select masteridto::int from working.sip_projs))
  and masteridto not in ((select masteridfrom::int from working.sip_projs)
                            union 
                           (select masteridto::int from working.sip_projs));
               
grant all on working.speed_hump_control_group to public;               
select * from working.speed_hump_control_group


""".format(study_year_min=study_year_min,study_year_max=study_year_max) 
)


In [70]:
control_group

,mft,masteridfrom,masteridto,segmentid,nodeidfrom,nodeidto
0,16,14,14,16,14,16
1,20,14,19,20,19,14
2,26,14,22,26,22,14
3,28,22,23,28,23,22
4,29,23,25,29,25,23
5,30,19,26,30,26,19
6,31,22,26,31,26,22
7,32,25,27,32,27,25
8,33,26,27,33,27,26
9,61,51,53,61,51,53


In [71]:
cdb.connect()
cdb.dataframe_to_table_schema(control_group, 'speed_hump_control_group', schema='working', overwrite = True)

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2019-12-27 17:42:48


[['mft', 'bigint'],
 ['masteridfrom', 'bigint'],
 ['masteridto', 'bigint'],
 ['segmentid', 'bigint'],
 ['nodeidfrom', 'bigint'],
 ['nodeidto', 'bigint']]

In [72]:
cdb.connect()
input_schema = cdb.dataframe_to_table_schema(control_group, 'speed_hump_control_group', schema='working', overwrite = True)
cdb.dataframe_to_table(control_group, 'speed_hump_control_group', input_schema, schema= "working")

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2019-12-27 17:42:50
Reading data into Database



50557it [17:32, 44.81it/s]

OperationalError: FATAL:  LDAP authentication failed for user "soge"


50557it [17:50, 44.81it/s]

### Control Group Nodes


In [69]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-01-16 12:15:56


In [75]:
control_group_nodes = cdb.dfquery("""


select distinct nodeidfrom "NODEID" from working.speed_hump_control_group

union 

select distinct nodeidto "NODEID" from working.speed_hump_control_group



"""
)

In [76]:
control_group_nodes

,NODEID
0,79030
1,26264
2,9002141
3,37876
4,55864
5,40694
6,9004435
7,47114
8,48663
9,47216


In [25]:
control_group_nodes.to_csv('control_group_nodes.csv')

In [93]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2019-12-30 17:01:12


In [35]:
inclusive_control_group = cdb.dfquery("""




--Sip control group

--drop table if exists inclusive_group; 

--create table inclusive_group as 

with data as (


select distinct coalesce(lion.mft,0) mft, coalesce(lion.masteridfrom,0) masteridfrom, coalesce(lion.masteridto,0) masteridto, lion.geom
from v_clean_speed_humps sh
join archive."18d.2019-11-13_lion" lion
on sh.segmentid = lion.segmentid 
where left(date_installed::text, 4)::int between {study_year_min} and {study_year_max}

)



select distinct mft, masteridfrom, masteridto, segmentid::int, nodeidfrom::int, nodeidto::int
from archive."18d.2019-11-13_lion" lion
where mft in (select mft from data)
  or masteridfrom in ((select masteridfrom mid from data)
                            union 
                           (select masteridto mid from data))
  or masteridto in   ((select masteridfrom mid from data)
                            union 
                           (select masteridto mid from data))
  or mft in (select mft::int from working.sip_projs)
  or masteridfrom in ((select masteridfrom::int from working.sip_projs)
                            union 
                           (select masteridto::int from working.sip_projs))
  or masteridto in ((select masteridfrom::int from working.sip_projs)
                            union 
                           (select masteridto::int from working.sip_projs));
               
--grant all on working.speed_hump_control_group to public;               
--select * from working.speed_hump_control_group

""".format(study_year_min=study_year_min,study_year_max=study_year_max) 
)


In [36]:
inclusive_control_group

,mft,masteridfrom,masteridto,segmentid,nodeidfrom,nodeidto
0,266,178,184,266,184,178
1,267,184,186,267,184,185
2,270,186,188,270,186,188
3,271,184,184,271,187,184
4,272,183,188,272,188,183
5,273,188,236,273,188,236
6,274,186,186,268,185,182
7,274,186,186,269,186,185
8,274,186,186,274,189,186
9,280,184,193,280,193,184


In [5]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-01-02 12:37:56


In [6]:
inclusive_control_group = cdb.dfquery("""


select distinct nodeidfrom from inclusive_group

union 

select distinct nodeidto from inclusive_group

"""
)

Failure:

- Query run 2020-01-02 12:37:57.480000
	


select distinct nodeidfrom from inclusive_group

union 

select distinct nodeidto from inclusive_group




SystemExit: 

C:\Python27\ArcGIS10.4\lib\site-packages\IPython\core\interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### All Injuries by node from FORMS and TAMS Data

In [45]:
node_inj = sqldb.dfquery("""


select * from (
select  
c.NODEID, 
cast(year(c.ACCIDENT_DT) as int) YR,
/*sum(case when PED_NONPED = 'P' then 1 else 0 end) as [PED],
sum(case when PED_NONPED = 'B' then 1 else 0 end) as [BIKE],
sum(case when PED_NONPED = 'N' then 1 else 0 end) as [MVO]*/
'after' as PERIOD,
case when PED_NONPED = 'Pedestrian' then 'ped'
    when PED_NONPED = 'Bicyclist' then 'bicycle'
    when PED_NONPED = 'Occupant' then 'mvo' end as Mode,
sum(case when i.INJ_KILLED = 'Injured' then 1 else 0 end) as INJ
from [FORMS].[dbo].[WC_ACCIDENT_F] as c
join FORMS.dbo.WC_ACCIDENT_VICTIM_F as i
    on c.INTEGRATION_ID=i.ACCIDENT_ID
where cast(year(c.ACCIDENT_DT) as int) = 2017
    and coalesce(c.VOID_STATUS_CD , 'N') = 'N'
    and c.NONMV is null
group by c.NODEID, 
         cast(year(c.ACCIDENT_DT) as int),
         case when PED_NONPED = 'Pedestrian' then 'ped'
              when PED_NONPED = 'Bicyclist' then 'bicycle'
              when PED_NONPED = 'Occupant' then 'mvo' end 

union


Select 
NODEID,
year(OCCURRENCE_DATETIME) YR,
case when year(OCCURRENCE_DATETIME) between 2011 and 2013 then 'before'
when year(OCCURRENCE_DATETIME) between 2015 and 2017 then 'after' end as PERIOD,
case when PED_NONPED = 'P' then 'ped'
    when PED_NONPED = 'B' then 'bicycle'
    when PED_NONPED = 'N' then 'mvo' end as Mode,
/*sum(case when PED_NONPED = 'P' then 1 else 0 end) as [PED],
sum(case when PED_NONPED = 'B' then 1 else 0 end) as [BIKE],
sum(case when PED_NONPED = 'N' then 1 else 0 end) as [MVO],
year(OCCURRENCE_DATETIME) YR*/
sum(INJURED_COUNT) as INJ
From(
    SELECT  c.ANUM_PCT
    , c.ANUM_SEQ
    , c.ANUM_YY
    , c.OCCURRENCE_DATETIME
    , c.NODEID
    , c.INJURED_COUNT
    , c.KILLED_COUNT
    , v.PED_NONPED
    From DataWarehouse.dbo.AIS_PD_Core_F as c
    join DataWarehouse.dbo.AIS_PD_Victim_F as v
    on c.ANUM_PCT = v.ANUM_PCT and c.ANUM_SEQ = v.ANUM_SEQ and c.ANUM_YY = v.ANUM_YY
    join DataWarehouse.dbo.AIS_PD_Locx_F as l
    on c.ANUM_PCT=l.ANUM_PCT and c.ANUM_SEQ = l.ANUM_SEQ and c.ANUM_YY = l.ANUM_YY
    where year(c.OCCURRENCE_DATETIME) between 2011 and 2016
    and c.NODEID in (select distinct coalesce(Nodeid,0) from
                     DataWarehouse.dbo.AIS_PD_Core_F)
            )x
    group by NODEID, 
             case when PED_NONPED = 'P' then 'ped'
                  when PED_NONPED = 'B' then 'bicycle'
                  when PED_NONPED = 'N' then 'mvo' end,
             year(OCCURRENCE_DATETIME)
             ) inj_data
where YR != 2014



""")

	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly



In [47]:
sqldb.connect()

	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly

Database connection (MS) to forms on dot55sql01 - user: arcgis 
Connection established 2020-01-16 11:19:06


In [51]:
node_inj = sqldb.dfquery("""



SELECT NODEID
      ,CASE WHEN yr between 2011 and 2013 THEN 'before' 
            WHEN yr between 2015 and 2017 THEN 'after' ELSE 'exclude' END AS PERIOD
      ,sum(INJ) INJURIES
FROM (  
        SELECT nodeid
              ,yr
              ,CASE WHEN PED_NONPED = 'Pedestrian' THEN 'ped'
                 WHEN PED_NONPED = 'Bicyclist' THEN 'bicycle'
                 WHEN PED_NONPED = 'Occupant' THEN 'mvo' END as Mode
              ,sum(CASE WHEN INJ_KILLED = 'Injured' THEN 1 else 0 END) AS INJ
        FROM (SELECT coalesce(c.nodeid, lion_node_number) nodeid
                    ,year(c.accident_dt) yr
                    ,ped_nonped
                    ,inj_killed 
              FROM [FORMS].[dbo].wc_accident_f c
              LEFT JOIN FORMS.dbo.WC_ACCIDENT_VICTIM_F AS i
              ON c.INTEGRATION_ID=i.ACCIDENT_ID
              WHERE year(c.ACCIDENT_DT) > 2016 
                and coalesce(c.VOID_STATUS_CD , 'N') = 'N'
                and coalesce(nonmv, 0) = 0
                and coalesce(nodeid, lion_node_number) is not null 
              --unioning all crashes with null nodeid

              UNION

              SELECT s.nodeid
                    ,year(c.accident_dt) yr
                    ,ped_nonped
                    ,inj_killed  
              FROM [FORMS].[dbo].wc_accident_f c
              JOIN [FORMS].[dbo].[v_IntersectionStreetNames_Gen] s
              ON lower(ltrim(rtrim(c.SRC_ON_STREET))) = lower(ltrim(rtrim(s.street_1))) 
              and
              lower(ltrim(rtrim(c.SRC_cross_STREET))) = lower(ltrim(rtrim(s.street_2)))
              and 
              CASE WHEN C.[SRC_POLICE_PRECINCT] BETWEEN 1 AND 34 THEN 1
                   WHEN C.[SRC_POLICE_PRECINCT] BETWEEN 40 AND 52 THEN 2
                   WHEN C.[SRC_POLICE_PRECINCT] BETWEEN 60 AND 94 THEN 3
                   WHEN C.[SRC_POLICE_PRECINCT] BETWEEN 100 AND 115 THEN 4
                   WHEN C.[SRC_POLICE_PRECINCT] BETWEEN 120 AND 123 THEN 5 END 
              in (left(b7sc_2, 1), left(b7sc_1, 1))
              LEFT JOIN FORMS.dbo.WC_ACCIDENT_VICTIM_F AS i
              ON c.INTEGRATION_ID=i.ACCIDENT_ID
              WHERE year(c.ACCIDENT_DT) > 2016
                and coalesce(c.VOID_STATUS_CD , 'N') = 'N'
                and coalesce(nonmv, 0) = 0 
                and coalesce(c.nodeid, lion_node_number) is null
        ) all_forms
        GROUP BY nodeid, yr,
        CASE WHEN PED_NONPED = 'Pedestrian' THEN 'ped'
             WHEN PED_NONPED = 'Bicyclist' THEN 'bicycle'
             WHEN PED_NONPED = 'Occupant' THEN 'mvo' END

        UNION 

        SELECT crashes.nodeid
              ,crashes.yr
              ,CASE WHEN v.PED_NONPED = 'P' THEN 'ped'
                    WHEN v.PED_NONPED = 'B' THEN 'bicycle'
                    WHEN v.PED_NONPED = 'N' THEN 'mvo' 
                    ELSE 'STUDY YEAR' END AS Mode 
              ,coalesce(sum(v.INJURED_COUNT),0) INJ
        FROM(--selecting tams crashes
             SELECT DISTINCT c.nodeid
                            ,c.anum_pct
                            ,c.anum_seq
                            ,c.anum_yy
                            ,yr
             FROM (SELECT nodeid
                          ,anum_pct
                          ,anum_seq
                          ,anum_yy
                          ,year(occurrence_datetime) yr
                    FROM DataWarehouse.dbo.AIS_PD_Core_F  
                    WHERE (year(occurrence_datetime) between 2011 and 2016)
                       and year(occurrence_datetime) != 2014
                       and nodeid > 0
                    ) AS c
            JOIN DataWarehouse.dbo.AIS_PD_Locx_F AS l
            ON c.ANUM_PCT = l.ANUM_PCT and c.ANUM_SEQ = l.ANUM_SEQ and c.ANUM_YY = l.ANUM_YY

            UNION

            --unioning to account for tams crashes with nodeid = 0/street name combos with multiple potential nodes
            SELECT DISTINCT lu.nodeid, c.anum_pct, c.anum_seq, c.anum_yy, yr
            FROM (SELECT anum_pct, anum_seq, anum_yy, year(occurrence_datetime) yr 
                  FROM DataWarehouse.dbo.AIS_PD_Core_F 
                  WHERE (year(occurrence_datetime) between 2011 and 2016)
                     and year(occurrence_datetime) != 2014
                  ) AS c
            JOIN DataWarehouse.dbo.AIS_PD_Locx_F AS l
            ON c.ANUM_PCT = l.ANUM_PCT and c.ANUM_SEQ = l.ANUM_SEQ and c.ANUM_YY = l.ANUM_YY
            JOIN DataWarehouse.dbo.nypd_location_lookup_F AS lu
            ON c.ANUM_PCT = lu.ANUM_PCT and l.ON_STREET in (lu.[ON_STREET], lu.CROSS_STREET) and l.CROSS_STREET in (lu.[ON_STREET], lu.CROSS_STREET)
            ---AG changed above from: on c.ANUM_PCT = lu.ANUM_PCT and l.ON_STREET = lu.[ON_STREET] and l.CROSS_STREET = lu.CROSS_STREET
        ) crashes
        LEFT JOIN DataWarehouse.dbo.AIS_PD_Victim_F AS v
        on crashes.ANUM_PCT = v.ANUM_PCT and crashes.ANUM_SEQ = v.ANUM_SEQ and crashes.ANUM_YY = v.ANUM_YY
        GROUP BY crashes.nodeid
                ,crashes.yr
                ,CASE WHEN v.PED_NONPED = 'P' THEN 'ped'
                      WHEN v.PED_NONPED = 'B' THEN 'bicycle'
                      WHEN v.PED_NONPED = 'N' THEN 'mvo' 
                      ELSE 'STUDY YEAR' END 

        ) data 

GROUP BY nodeid, 
       CASE WHEN yr between 2011 and 2013 THEN 'before' 
            WHEN  yr between 2015 and 2017 THEN 'after' ELSE 'exclude' END
""")


	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly



In [31]:
 study_group_nodes

,NODEID
0,52768
1,9001246
2,9001310
3,46708
4,18803
5,62993
6,42179
7,73888
8,74939
9,59388


In [53]:
x = study_group_nodes.merge(node_inj, on='NODEID', how='left')

In [82]:
x.loc[x['PERIOD'] == 'after'].INJURIES.sum()

978.0

In [83]:
x.loc[x['PERIOD'] == 'before'].INJURIES.sum()

1224.0

In [77]:
y = control_group_nodes.merge(node_inj, on='NODEID', how='left')

In [86]:
y.loc[y['PERIOD'] == 'after'].INJURIES.sum()

79235.0

In [87]:
y.loc[y['PERIOD'] == 'before'].INJURIES.sum()

95819.0